In [1]:
import numpy
import glob
import ast

import matplotlib.pyplot as plt
from math import isclose

import mozaik
from parameters import ParameterSet
from mozaik.storage.datastore import PickledDataStore
from mozaik.storage.queries import *
from mozaik.analysis.analysis import *
from mozaik.analysis.vision import *
from mozaik.meta_workflow.parameter_search import ParameterSearch
from json_export import save_json
from bluepyopt.deapext.optimisationsCMA import DEAPOptimisationCMA

from utils import *

In [2]:
from evaluator import define_evaluator

evaluator = define_evaluator(
    run_script="run_experiment.py",
    parameters_url="param/defaults",
    timeout = 2000
)

In [3]:
import psutil
process = psutil.Process()


for data_store in get_data_stores("20241025-040251"):
    print()
    print("Before: ", process.memory_info().rss / 1024**3) 
    score = evaluator.evaluate_data_store(data_store)
    print("After: ", process.memory_info().rss / 1024**3) 
    print(score)

Reading folder ./20241025-040251_Optimization/SelfSustainedPushPull_Opt_174300_____experiment_scaler_a:109.99999999_experiment_scaler_b:0.91106588
Reading folder ./20241025-040251_Optimization/SelfSustainedPushPull_Opt_174303_____experiment_scaler_a:105.80476743440181_experiment_scaler_b:0.91
Reading folder ./20241025-040251_Optimization/SelfSustainedPushPull_Opt_174305_____experiment_scaler_a:109.99999999_experiment_scaler_b:0.91999999
Reading folder ./20241025-040251_Optimization/SelfSustainedPushPull_Opt_174308_____experiment_scaler_a:109.99999999_experiment_scaler_b:0.84632001
Reading folder ./20241025-040251_Optimization/SelfSustainedPushPull_Opt_174302_____experiment_scaler_a:109.99999999_experiment_scaler_b:0.91999999
Reading folder ./20241025-040251_Optimization/SelfSustainedPushPull_Opt_174301_____experiment_scaler_a:108.10238163535846_experiment_scaler_b:0.88
Reading folder ./20241025-040251_Optimization/SelfSustainedPushPull_Opt_174299_____experiment_scaler_a:105.35489805473

/home/tanguy.damart/virt_env/mozaik/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/tanguy.damart/virt_env/mozaik/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/tanguy.damart/virt_env/mozaik/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/tanguy.damart/virt_env/mozaik/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/tanguy.damart/virt_env/mozaik/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


For feature Irregularity_V1_Exc_L4, for value 0.7822 computed score 2.18


/home/tanguy.damart/virt_env/mozaik/lib/python3.8/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/tanguy.damart/virt_env/mozaik/lib/python3.8/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


For feature Synchrony_V1_Exc_L4, for value 0.0019 computed score 0.19


NeuronAnnotationsToPerNeuronValues analysis took: 0.3331880569458008 seconds


For feature OrientationTuningPreference_V1_Exc_L4, for value 3.3241 dimensionless computed score 3.35 dimensionless
For feature OrientationTuningOrthoHigh_V1_Exc_L4, for value 0.1462 dimensionless computed score 0.29 dimensionless


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f1b6e628250>>
Traceback (most recent call last):
  File "/home/tanguy.damart/virt_env/mozaik/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
run_id = "20241023-064522"
sheet_names = ["V1_Exc_L4", "V1_Inh_L4", "V1_Exc_L2/3", "V1_Inh_L2/3"]
sheet_name = sheet_names[0]
data_store = get_data_stores(run_id)[0]

In [ ]:
# CC AND CV
spiketrains = param_filter_query(data_store, sheet_name=sheet_name, st_name='InternalStimulus').get_segments()[0].spiketrains
t_start = round(spiketrains[0].t_start, 5)
t_stop = round(spiketrains[0].t_stop, 5)
num_bins = int(round((t_stop - t_start) / 10.))
r = (float(t_start), float(t_stop))
psths = [numpy.histogram(x, bins=num_bins, range=r)[0] for x in spiketrains[:1000]]
corrs = numpy.nan_to_num(numpy.corrcoef(numpy.squeeze(psths)))
isis = [numpy.diff(st.magnitude) for st in spiketrains[:1000]]
idxs = numpy.array([len(isi) for isi in isis]) > 5
print("CC: ", numpy.mean(corrs[idxs, :][:, idxs][numpy.triu_indices(sum(idxs == True), 1)]))

In [ ]:
# Choose the O to consider
target_O = numpy.pi / 2
orthogonal_O = target_O - (numpy.pi / 2)

# Get the id of the neurons that respond to that O
orientations_cells = get_orientation_preference(data_store, sheet_name).values
idx_cells = [idx for idx, oc in enumerate(orientations_cells) if isclose(target_O, oc, abs_tol=0.1)]

# Get the spontaneous rate
seg = param_filter_query(data_store, sheet_name=sheet_name, st_name='InternalStimulus').get_segments()[0]
print(len(seg.spiketrains), len(orientations_cells))
spont_rate = get_mean_rate(seg.spiketrains, idx_cells)

segments = queries.param_filter_query(data_store, st_name='FullfieldDriftingSinusoidalGrating', sheet_name=sheet_name).get_segments()
for seg in segments:
    spiketrains = seg.spiketrains
    print(get_annotation(seg, "orientation"), get_annotation(seg, "contrast"), numpy.mean([len(spiketrains[idx]) for idx in idx_cells]) / get_spiketrains_duration(spiketrains))

# Get the rates for the O and the ortho O
segments = queries.param_filter_query(data_store, st_name='FullfieldDriftingSinusoidalGrating', sheet_name=sheet_name, st_contrast=[100]).get_segments()
for seg in segments:
    orientation = get_annotation(seg, "orientation")
    if isclose(orientation, target_O, abs_tol=0.1):
        spike_rate_high = get_mean_rate(seg.spiketrains, idx_cells)
    elif isclose(orientation, orthogonal_O, abs_tol=0.1):
        spike_rate_ortho_high = get_mean_rate(seg.spiketrains, idx_cells)

segments = queries.param_filter_query(data_store, st_name='FullfieldDriftingSinusoidalGrating', sheet_name=sheet_name, st_contrast=[10]).get_segments()
for seg in segments:
    orientation = get_annotation(seg, "orientation")
    if isclose(orientation, orthogonal_O, abs_tol=0.1):
        spike_rate_ortho_low = get_mean_rate(seg.spiketrains, idx_cells)
        break

In [ ]:
# Compute the feature values
print(spont_rate)
print(spike_rate_high)
print(spike_rate_ortho_high)
print(spike_rate_ortho_low)

value_1 = spike_rate_high / spike_rate_ortho_high
value_2 = numpy.abs(spont_rate - spike_rate_ortho_high) / spont_rate
value_3 = numpy.abs(spont_rate - spike_rate_ortho_low) / spont_rate

print(value_1)
print(value_2)
print(value_3)

In [ ]:
# Test target ICMS: one per amplitude ? aggregate the sheet and distances and 3 features into one target

run_id = "20241018-050106"
sheet_names = ["V1_Exc_L4", "V1_Inh_L4", "V1_Exc_L2/3", "V1_Inh_L2/3"]
selection_radius = 200.
stimulation_frequency = 100
stimulation_ISI = 1000 / stimulation_frequency
n_electrodes = 100

data_stores = get_data_stores(run_id)

activity_per_electrode = [[] for i in range(n_electrodes)]
for data_store in data_stores:
    for sheet_name in sheet_names:

        data_view_ICMS = param_filter_query(data_store, sheet_name=sheet_name, st_direct_stimulation_name="IntraCorticalMicroStimulation",)
        data_spike_ICMS = data_view_ICMS.get_segments()[0].spiketrains

        probe_positions, cells_positions, electrode_active_per_cell = get_ICMS_position_data(sheet_name, data_store)
        cells_positions = cells_positions.T
        pair_distances = cdist(probe_positions.values[:, :2], probe_positions.values[:, :2])
        distances = cdist(probe_positions.values, cells_positions.values)

        for electrode_idx in range(n_electrodes):
            for cell_idx in range(len(cells_positions.values)):
                if distances[electrode_idx, cell_idx] < selection_radius:
                    _spikes = numpy.array(data_spike_ICMS[cell_idx])
                    activity_per_electrode[electrode_idx] += list(
                        ((_spikes[_spikes > stimulation_ISI]) % stimulation_ISI)
                    )

        del data_view_ICMS
        del data_spike_ICMS
        gc.collect()

excitation_amplitudes = []
inhibition_durations = []
inhibition_depths = []

for electrode_idx in range(n_electrodes):

    assert numpy.max(activity_per_electrode[electrode_idx]) <= stimulation_ISI
    assert numpy.min(activity_per_electrode[electrode_idx]) >= 0

    hist_raw, bin_edges = numpy.histogram(activity_per_electrode[electrode_idx], bins=nbins, range=(0, stimulation_ISI))
    mean_rate = numpy.mean(hist_raw[nbins_spont_skip:])
    hist_normed = (hist_raw - mean_rate) / mean_rate

    excitation_amplitudes.append(numpy.max(hist_normed[:2]))

    # Compute the length of the inhibitory period
    tmp_smoothed = numpy.convolve(hist_normed, numpy.ones(3) / 3, mode='valid')
    hist_smooth = numpy.copy(hist_normed)
    hist_smooth[1:-1] = tmp_smoothed

    inh_start_bin = next((bin_idx for bin_idx in range(4) if hist_smooth[bin_idx] < threshold_inhibition), None)

    if inh_start_bin is not None:
        inh_end_bin = inh_start_bin + 1
        while inh_end_bin < len(hist_smooth) and hist_smooth[inh_end_bin] < threshold_inhibition: inh_end_bin += 1
        inhibition_durations.append((inh_end_bin - inh_start_bin) * bin_with)
        inhibition_depths.append(numpy.min(hist_smooth[inh_start_bin:inh_end_bin + 1]))
    else:
        inhibition_durations.append(0.)
        inhibition_depths.append(0)

    # Plot PSTH
    if electrode_idx in [35, 45]:
        fig, axs = plot_PSTH(electrode_idx, stimulation_amplitude, excitation_amplitudes[-1], inhibition_durations[-1],
                             inhibition_depths[-1], hist_raw, hist_normed, hist_smooth, bin_edges, mean_rate, stimulation_ISI)
        display(fig)
        plt.clf()

    export_data[stimulation_amplitude][0].append(pair_distances[active_electrode, electrode_idx])
    export_data[stimulation_amplitude][1].append(excitation_amplitudes[-1])
    export_data[stimulation_amplitude][2].append(inhibition_durations[-1])
    export_data[stimulation_amplitude][3].append(inhibition_depths[-1])

# Plot view of the array
fig, axs = plot_array_view(excitation_amplitudes, inhibition_durations, inhibition_depths, run_id, stimulation_amplitude)
display(fig)
plt.clf()

_plot_data_reshape = numpy.array(excitation_amplitudes).reshape(10, 10) + 1
fig, ax = plt.subplots(1, figsize=(12, 12.8), dpi=50)
sns.heatmap(_plot_data_reshape, xticklabels=False, yticklabels=False,
            annot=numpy.arange(0, 100, 1).reshape(10, 10), 
            cmap='Blues',
            linewidths=1, ax=ax, cbar=False, annot_kws={"fontsize": 30},
            vmin=1, vmax=4, rasterized=False)
plt.tight_layout()
plt.savefig(f"figures/{run_id}__{stimulation_amplitude}uA__topview_exc.png",
            dpi=250, bbox_inches='tight', transparent=True, pad_inches=0)
plt.show()
plt.clf()

assert len(export_data[stimulation_amplitude][0]) == len(export_data[stimulation_amplitude][1])

fig, ax = plt.subplots(3, figsize=(4, 6))
for l in range(3):
    ax[l].scatter(export_data[stimulation_amplitude][0], export_data[stimulation_amplitude][l + 1], color='black', )
ax[2].set_xlabel("Distance to stimulating electrode ($\mu$m)")
plt.show()
plt.clf()

del activity_per_electrode
del data_stores
gc.collect()

In [ ]:
from scipy.spatial.distance import cdist

class ICMSTarget():

    def __init__(self, name, target_values, value_distances, sheet_names, active_electrode, stimulation_frequency, amplitude, norms=None, max_score=10):
        self.name  = name
        self.value_distances = value_distances
        self.target_values = target_values
        self.norms = norms
        self.max_score = max_score
        self.sheet_names = sheet_names
        self.active_electrode = active_electrode
        self.stimulation_frequency = stimulation_frequency
        self.n_electrodes = 100
        self.amplitude = amplitude
        assert all(n != 0 for n in self.norms)

    def _get_activity_per_electrode(self):

        activity_per_electrode = [[] for i in range(self.n_electrodes)]
        selection_radius = 200.
        stimulation_ISI = 1000. / self.stimulation_frequency

        for sheet_name in self.sheet_names:

            data_view_ICMS = param_filter_query(
                data_store, sheet_name=sheet_name, st_direct_stimulation_name="IntraCorticalMicroStimulation"
            )
            data_spike_ICMS = data_view_ICMS.get_segments()[0].spiketrains

            probe_positions, cells_positions, electrode_active_per_cell = get_ICMS_position_data(
                sheet_name, data_store, amplitude=self.amplitude, frequency=self.stimulation_frequency, active_electrodes=[self.active_electrode]
            )
            cells_positions = cells_positions.T
            pair_distances = cdist(probe_positions.values[:, :2], probe_positions.values[:, :2])
            distances = cdist(probe_positions.values, cells_positions)

            for electrode_idx in range(self.n_electrodes):
                for cell_idx in range(len(cells_positions)):
                    if distances[electrode_idx, cell_idx] < selection_radius:
                        _spikes = numpy.array(data_spike_ICMS[cell_idx])
                        activity_per_electrode[electrode_idx] += list(
                            ((_spikes[_spikes > stimulation_ISI]) % stimulation_ISI)
                        )

            del data_view_ICMS
            del data_spike_ICMS
            gc.collect()

        return activity_per_electrode

    def _order_by_exp_distances(self, values_sim, distances_sim):
        ordered_values = [[] for i in range(len(self.value_distances))]
        for idx_exp, dist_exp in enumerate(self.value_distances):
            for idx_sim, dist_sim in enumerate(distances_sim):
                if numpy.isclose(dist_exp, dist_sim, atol=1):
                    ordered_values[idx_exp].append(values_sim[idx_sim])
        ordered_values = [numpy.mean(ov) if ov else None for ov in ordered_values]
        return ordered_values

    def calculate_values(self, data_store):
        raise NotImplementedError

    def calculate_score(self, data_store):
        if data_store is None:
            return self.max_score
        values = self.calculate_values(data_store)
        print(values)
        if values is None:
            scores = self.max_score
        else:
            scores = []
            for exp_value, norm, sim_value, dist in zip(self.target_values, self.norms, values, self.value_distances):
                if sim_value is not None and dist < 2000:
                    #print(exp_value, sim_value, norm, abs(exp_value - sim_value) / norm)
                    scores.append(abs(exp_value - sim_value) / norm)
                    plt.scatter(dist, exp_value, color="C0")
                    plt.scatter(dist, sim_value, color="C1")
                    
            score = numpy.sum(scores)
            plt.show()
            plt.clf()
        print(score)
        threshold_score = min(score, self.max_score)
        return threshold_score


class ICMSExcAmplitudeTarget(ICMSTarget):
    def calculate_values(self, data_store):

        activity_per_electrode = self._get_activity_per_electrode()

        probe_positions, cells_positions, electrode_active_per_cell = get_ICMS_position_data(
            self.sheet_names[0], data_store, amplitude=self.amplitude, frequency=self.stimulation_frequency, active_electrodes=[self.active_electrode]
        )
        pair_distances = cdist(probe_positions.values[:, :2], probe_positions.values[:, :2])

        excitation_amplitudes = []
        distances = []

        bin_with = 20 # ms
        spont_skip = 1000 # ms
        stimulation_ISI = 1000. / self.stimulation_frequency
        nbins = round(stimulation_ISI / bin_with)
        nbins_spont_skip = round((stimulation_ISI - spont_skip) / bin_with)

        # Compute the values
        for electrode_idx in range(self.n_electrodes):
            hist_raw, bin_edges = numpy.histogram(activity_per_electrode[electrode_idx], bins=nbins, range=(0, stimulation_ISI))
            mean_rate = numpy.mean(hist_raw[nbins_spont_skip:])
            hist_normed = (hist_raw - mean_rate) / mean_rate
            excitation_amplitudes.append(numpy.max(hist_normed[:2]))
            distances.append(pair_distances[self.active_electrode, electrode_idx])

        # Order by distances of the exp data
        return self._order_by_exp_distances(excitation_amplitudes, distances)


class ICMSInhDurationTarget(ICMSTarget):
    def calculate_values(self, data_store):

        activity_per_electrode = self._get_activity_per_electrode()

        probe_positions, cells_positions, electrode_active_per_cell = get_ICMS_position_data(
            self.sheet_names[0], data_store, amplitude=self.amplitude, frequency=self.stimulation_frequency, active_electrodes=[self.active_electrode]
        )
        pair_distances = cdist(probe_positions.values[:, :2], probe_positions.values[:, :2])

        inhibition_durations = []
        distances = []

        bin_with = 20 # ms
        spont_skip = 1000 # ms
        stimulation_ISI = 1000. / self.stimulation_frequency
        nbins = round(stimulation_ISI / bin_with)
        nbins_spont_skip = round((stimulation_ISI - spont_skip) / bin_with)

        # Compute the values
        for electrode_idx in range(self.n_electrodes):
            hist_raw, bin_edges = numpy.histogram(activity_per_electrode[electrode_idx], bins=nbins, range=(0, stimulation_ISI))
            mean_rate = numpy.mean(hist_raw[nbins_spont_skip:])
            hist_normed = (hist_raw - mean_rate) / mean_rate

            # Compute the length of the inhibitory period
            tmp_smoothed = numpy.convolve(hist_normed, numpy.ones(3) / 3, mode='valid')
            hist_smooth = numpy.copy(hist_normed)
            hist_smooth[1:-1] = tmp_smoothed
        
            inh_start_bin = next((bin_idx for bin_idx in range(4) if hist_smooth[bin_idx] < 0.1), None)
        
            if inh_start_bin is not None:
                inh_end_bin = inh_start_bin + 1
                while inh_end_bin < len(hist_smooth) and hist_smooth[inh_end_bin] < 0.1: inh_end_bin += 1
                inhibition_durations.append((inh_end_bin - inh_start_bin) * bin_with)
            else:
                inhibition_durations.append(0.)

            distances.append(pair_distances[self.active_electrode, electrode_idx])

        # Order by distances of the exp data
        return self._order_by_exp_distances(inhibition_durations, distances)


class ICMSInhDepthTarget(ICMSTarget):
    def calculate_values(self, data_store):

        activity_per_electrode = self._get_activity_per_electrode()

        probe_positions, cells_positions, electrode_active_per_cell = get_ICMS_position_data(
            self.sheet_names[0], data_store, amplitude=self.amplitude, frequency=self.stimulation_frequency, active_electrodes=[self.active_electrode]
        )
        pair_distances = cdist(probe_positions.values[:, :2], probe_positions.values[:, :2])

        inhibition_depths = []
        distances = []

        bin_with = 20 # ms
        spont_skip = 1000 # ms
        stimulation_ISI = 1000. / self.stimulation_frequency
        nbins = round(stimulation_ISI / bin_with)
        nbins_spont_skip = round((stimulation_ISI - spont_skip) / bin_with)

        # Compute the values
        for electrode_idx in range(self.n_electrodes):
            hist_raw, bin_edges = numpy.histogram(activity_per_electrode[electrode_idx], bins=nbins, range=(0, stimulation_ISI))
            mean_rate = numpy.mean(hist_raw[nbins_spont_skip:])
            hist_normed = (hist_raw - mean_rate) / mean_rate

            # Compute the length of the inhibitory period
            tmp_smoothed = numpy.convolve(hist_normed, numpy.ones(3) / 3, mode='valid')
            hist_smooth = numpy.copy(hist_normed)
            hist_smooth[1:-1] = tmp_smoothed

            inh_start_bin = next((bin_idx for bin_idx in range(4) if hist_smooth[bin_idx] < 0.1), None)

            if inh_start_bin is not None:
                inh_end_bin = inh_start_bin + 1
                while inh_end_bin < len(hist_smooth) and hist_smooth[inh_end_bin] < 0.1: inh_end_bin += 1
                inhibition_depths.append(numpy.min(hist_smooth[inh_start_bin:inh_end_bin + 1]))
            else:
                inhibition_depths.append(0.)

            distances.append(pair_distances[self.active_electrode, electrode_idx])

        # Order by distances of the exp data
        return self._order_by_exp_distances(inhibition_depths, distances)

In [ ]:
# Test it:
from evaluator import *
import pickle

# Find a datastore with ICMS
run_id = "20241024-051112"

for data_store in get_data_stores(run_id):

    print("\n\n\n\n")

    # Create an objective with ICMSExcAmplitudeTarget
    with open(f"ICMS_exp_data.pickle", 'rb') as fp:
        exp_data = pickle.load(fp)
    
    value_distances = list(exp_data[41][0][0])
    target_values = list(exp_data[41][0][1])
    norms = [0.75 for i in range(len(value_distances))]
    sheet_names = ["V1_Exc_L4", "V1_Inh_L4", "V1_Exc_L2/3", "V1_Inh_L2/3"]
    objective = ICMSExcAmplitudeTarget(
        "ICMSExcAmplitudeTarget", target_values, value_distances, sheet_names,
        active_electrode=45, stimulation_frequency=0.5, amplitude=41000.0, norms=norms, max_score=10
    )
    score = objective.calculate_score(data_store)
    print(score)

    value_distances = list(exp_data[41][1][0])
    target_values = list(exp_data[41][1][1])
    norms = [150 for i in range(len(value_distances))]
    sheet_names = ["V1_Exc_L4", "V1_Inh_L4", "V1_Exc_L2/3", "V1_Inh_L2/3"]
    objective = ICMSInhDurationTarget(
        "ICMSInhDurationTarget", target_values, value_distances, sheet_names,
        active_electrode=45, stimulation_frequency=0.5, amplitude=41000.0, norms=norms, max_score=10
    )
    score = objective.calculate_score(data_store)
    print(score)
    
    value_distances = list(exp_data[41][2][0])
    target_values = list(exp_data[41][2][1])
    norms = [0.75 for i in range(len(value_distances))]
    sheet_names = ["V1_Exc_L4", "V1_Inh_L4", "V1_Exc_L2/3", "V1_Inh_L2/3"]
    objective = ICMSInhDepthTarget(
        "ICMSInhDepthTarget", target_values, value_distances, sheet_names,
        active_electrode=45, stimulation_frequency=0.5, amplitude=41000.0, norms=norms, max_score=10
    )
    score = objective.calculate_score(data_store)
    print(score)